# data collection

Collect lots of things from lots of sources.




## basic http tools

Copying files from many sources.


### copy list of links to json

https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=400

Crime reports?


#### the code



In [ ]:
var fs = require('fs');
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crimes');

function scrapeAlert(ID) {
    if(fs.existsSync(path.join(project, 'maricopa-alert-' + ID + '.json'))) {
        return;
    }
    return client
        .url('https://www.maricopacountyattorney.org/CivicAlerts.aspx?AID=' + ID)
        .getAllXPath({
            time: '//*[@class = "single"]//*[@class = "date"]//text()',
            title: '//*[contains(@class, "single")]//h3//text()',
            content: '//*[@class = "single"]//*[@class = "content"]//text()'
        })
        .then(r => {
            fs.writeFileSync(path.join(project, 'maricopa-alert-' + ID + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = scrapeAlert;

if(typeof $$ !== 'undefined') {
    $$.async();
    var IDs = Array.from(Array(500).keys());
    multiCrawl(IDs, 'crime reports')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


### TODO: download amazon history

Because Amazon download offer it as a download like Netflix does.

https://www.amazon.com/gp/yourstore/iyr/ref=pd_ys_iyr_next?ie=UTF8&collection=watched&iyrGroup=&maxItem=616&minItem=600


### search a lot of engines




#### the code 

meta search all?


In [ ]:
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var multiCrawl = importer.import('multi crawl');

// http://www.exploratorium.edu/files/ronh/research/index.html
/*

 Exploratorium	
 Search
 Google	
 Search
 alltheweb	
 Search
 Teoma	
 Search
 AltaVista	
 Search
 Wisenut	
 Search
 HotBot	
 Search
 lii.org	
 Search
 Northern Light	
 Search
 Lycos	
 Search
 Scirus	
 
 Meta:
  Ask Jeeves	
 Search
 MetaCrawler	
 Search
 Dogpile	
 Search
 SavvySearch	
 
 
 */
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/searches');

function searchAll(query = 'search engine') {
    var engines = [
        'https://www.google.com/search?q=' + query,
        'https://www.bing.com/search?q=' + query,
        'https://search.yahoo.com/search?p=' + query,
        'https://www.ask.com/web?q=' + query,
        'https://search.aol.com/aol/search?q=' + query,
        'http://www.baidu.com/s?wd=' + query,
        'https://www.wolframalpha.com/input/?i=' + query,
        'https://duckduckgo.com/?q=' + query,
        'https://www.yandex.com/search/?text=' + query,
        'https://archive.org/search.php?query=' + query,
    ];
    
    // TODO: save results
    return multiCrawl(engines, 'search results json')
        .then(r => {
            const time = new Date();
            fs.writeFileSync(path.join(project, query.replace(/[^a-z0-9]/ig, '_')
                                       + '-' + time.getFullYear()
                                       + '-' + (time.getMonth() + 1)
                                       + '-' + time.getDate()
                                       + '.json'), JSON.stringify(r, null, 4));
            return r;
        })
}
module.exports = searchAll;

if(typeof $$ !== 'undefined') {
    $$.async();
    searchAll()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


#### schedule search all?

TODO: convert this to a pattern.



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search || 'search engines'
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('meta search all', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


#### tell joke?



In [ ]:
var util = require('bluebird');
var request = util.promisify(require('request'));
var importer = require('../Core')

var jokes;
function getJoke() {
    // TODO: collect jokes instead
    return (typeof jokes === 'undefined'
        ? request('http://www.ducksters.com/jokes/silly.php')
        .then(res => importer.regexToArray(/^.*?Q:.*$|^.*?A:.*$/igm, res.body))
        .then(r => {
            r = r.reduce((arr, j, i) => {
                if(i % 2 === 1) {
                    arr.push([
                        r[i-1].replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, ''),
                        j.replace(/<.*?\s*\/?>/ig, '').trim().replace(/^\s*|\s*$/igm, '')
                    ]);
                }
                return arr;
            }, []);
            console.log(r);
            jokes = r;
            return r;
        })
        : Promise.resolve(jokes))
        .then(arr => {
            const i = Math.round(Math.random() * arr.length);
            return arr[i];
        })
}
module.exports = getJoke;



## advanced http tools 



### TODO multicrawl with selenium


#### the code

multi crawl?


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

var TIMEOUT = 1000;
var CONNECTIONS = 3;

// recursively dequeue tasks
function deQueue(inputQueue, searchCallback, ctx) {
    const results = [];
    console.log(ctx.client.requestHandler.sessionID);
    const callback = typeof searchCallback === 'function'
        ? searchCallback
        : importer.import(searchCallback, Object.assign({useCache: false}, ctx));
    if(inputQueue.length > 0) {
        const item = inputQueue.shift();
        return new Promise(resolve => setTimeout(() => resolve(), 100))
            .then(() => callback(...[item, ctx]))
            .catch(e => {
                console.log(e + '');
                if((e + '').indexOf('Already') > -1 || (e + '').indexOf('session') > -1) {
                    inputQueue.push(item);
                    throw new Error('Abandoning session :(', e);
                }
            })
            .then(r => results.push(r))
            .then(() => deQueue(inputQueue, searchCallback, ctx))
            .then(r => results.concat(r))
            .catch(e => console.log(e))
    } else {
        return results;
    }
}

// create a number of individual selenium sessions and dequeue the tasks with the callback search
function multiCrawl(inputList, searchCallback) {
    var indexes = Array.from(Array(Math.min(inputList.length, CONNECTIONS)).keys());
    var connections = [];
    var promises = indexes.map((s, i) => resolve => {
        const client = runSeleniumCell(false, false);
        return client
            // skip this if error
            //.then(() => connections[i].onlyOneWindow())
            //.then(() => connections[i].resizeWindow())
            .then(ctx => {
                connections.push(ctx);
                resolve(ctx)
            })
            .catch(e => {
                console.log(e);
                resolve(null);
            })
    });
    var queue = [].concat(inputList);
    var count = 0;
    return importer.runAllPromises(promises)
        .then(() => {
            console.log(connections.map(c => c.client.requestHandler.sessionID));
        })
        .then(() => {
            return connections[0].client
                .scanning(true)
                .then(() => connections[0].getAllSessionUrls())
                .scanning(false)
        })
        .then(() => console.log('done loading sessions'))
        .then(() => Promise.all(connections.map(ctx => deQueue(queue, searchCallback, ctx))))
        .then(r => [].concat([], ...r))
}
module.exports = multiCrawl;



### crawl domain?



#### the code



In [ ]:
var {URL} = require('url')
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var {doBrowserRequest} = importer.import('browser crawler tools')
var {
    cacheFilename,
    existingCache,
    storeCache,
    readCache,
    rmhash
} = importer.import('domain cache tools')

async function crawlRecursive(url, depth, searches) {
    if(!depth) depth = 3 // TODO: minutes depth using time range?
    url = (typeof url === 'string' ? [url] : url )
    // searches2 keeps track of new pages that should be added if searches is not provided
    //   this guaruntees at least one page will be requested when this is called
    const searches2 = []
    for(var i = 0; i < url.length; i++) {
        var l = url[i]
        try {
            await doBrowserRequest(l, readCache.bind(null, searches || searches2),
                                   storeCache.bind(null, searches || searches2))
        } catch (e) {
            console.log(e)
        }
    }
    
    // TODO: fix this
    
    // push old cache on to bottom of current searches,
    //   so we are always getting at least the page requested
    if(!searches) searches = searches2.concat(existingCache(url[0]))
    
    var existing = searches.map(s => rmhash(s.url))
    var links = searches2
    // TODO: pattern defensive programming
        .map(s => {
            var styles = s.styles || []
            var links = s.links || []
            return styles.concat(links)
        })
        .flat()
        // do not include hash in actual link to the page
        .map(s => rmhash(s))
        // filter out first occurence
        .filter((l, i, arr) => arr.indexOf(l) === i
        // filter out all existing urls
                && !existing.includes(rmhash(l))
        // filter out data uris
                && !l.includes('data:') && !l.includes('mailto:')
                && !l.includes('javascript:') && !l.includes('ios-app:'))
    
    if(depth > 1) {
        return await crawlRecursive(links, depth - 1, searches)
    }
    
    // close the browser
    await doBrowserRequest(false)
    
    // save the database
    var filePath = cacheFilename(searches[0] ? searches[0].url : url[0])
    fs.writeFileSync(filePath, JSON.stringify(searches, null, 2))
}

async function crawlAll(url, depth, searches) {
    try {
        await crawlRecursive(url, depth, searches)
    } catch (e) {
        console.log(e)
        await doBrowserRequest(false)
    }
}

module.exports = crawlAll

//var importer = require('../Core')
//var crawlAll = importer.import('crawl domain')

if(typeof $$ !== 'undefined') {
    $$.async()
    crawlAll('https://google.com', 2)
        .then(r => $$.sendResult('done'))
        .catch(e => $$.sendError(e))
}



#### domain cache tools?



In [ ]:
var {URL} = require('url')
var fs = require('fs')
var path = require('path')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var {getResponseContent} = importer.import('browser crawler tools')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function cacheFilename(url) {
    if(typeof url === 'string')
        url = new URL(url.includes('://') ? url : ('http://' + url))
    const time = new Date()
    var file = safeurl(url.hostname)
       + '-' + time.getFullYear()
       + '-' + (time.getMonth() + 1)
       + '-' + time.getDate() + '.json'
    return path.join(project, file)
}

function findCache(url) {
    if(typeof url === 'string') {
        url = new URL(url.includes('://') ? url : ('http://' + url))
    }
    const host = safeurl(url.hostname)
    const crawl = glob(
        '**/*' + host + '*',
        project
    )
    crawl.sort((a, b) => {
        return fs.statSync(b).mtime.getTime() - fs.statSync(a).mtime.getTime()
    })
    return crawl
}

function existingCache(url) {
    var cache = findCache(url)
    var filePath = cacheFilename(url)
    // save pages from the same day in the same database using the url as the keys
    if(cache[0] === filePath) {
        return JSON.parse(fs.readFileSync(filePath)) || []
    }
    return []
}

async function storeCache(cache, response) {
    var headers = await response.headers()
    var result = await getResponseContent(response, headers)
    console.log('Received ' + result.url)
    cache.push(result)
}

async function readCache(cache, request) {
    const url = (await request.url()).toLowerCase()
    var response = cache.filter(s => s.url.toLowerCase().localeCompare(url) === 0)[0]
    if (response && response.expires
        && response.expires > Date.now()) {
        await request.respond({
            body: response.content,
            contentType: response.type
        })
        return
    }
    request.continue();
}

// TODO: move this to URL tools in Languages/html.ipynb with getAllLinks
function rmhash(url) {
    return url.replace(/#.*$/ig, '')
}

// TODO: replace other occurrences with this function
function safeurl(url) {
    return url.replace(/[^a-z0-9_-]/ig, '_').substr(0, 100)
}

module.exports = {
    cacheFilename,
    findCache,
    existingCache,
    storeCache,
    readCache,
    rmhash,
    safeurl,
}


#### browser crawler tools?

Partly derrived from https://help.apify.com/en/articles/2424032-cache-responses-in-puppeteer


In [ ]:
var puppeteer = require('puppeteer')
var importer = require('../Core')
var {selectDom} = importer.import('select tree')

// TODO: make a utility for this because it keeps coming up here and in convert spreadsheet
function getStyleUrls(content, url) {
    return importer.regexToArray(/url\s*\(['"]*([^\)]*?)['"]*\)/ig, content, 1)
        .map(l => new URL(l, url).href)
}

// TODO: make a utility for this because it keeps coming up here and in convert spreadsheet
function getAllLinks(url, doc) {
    var r = selectDom({
        links: ['//a/@href|(//img|//iframe|//audio)[@src]/@src'],
        styles: ['//link/@href|//style/@text()'],
        html: ['/*']
    }, doc)
    var html = r.html.map(h => h.outerHTML).join('')
    return {
        links: r.links.map(s => new URL(s, url).href)
            .concat(getStyleUrls(html, url)),
        styles: r.styles.map(s => new URL(s, url).href),
        html: html
    }
}

async function getExpires(headers) {
    const cacheControl = headers['cache-control'] || '';
    const maxAgeMatch = cacheControl.match(/max-age=(\d+)/);
    const maxAge = maxAgeMatch && maxAgeMatch.length > 1 ? parseInt(maxAgeMatch[1], 10) : 0;
    return Date.now() + (maxAge * 1000)
}

async function getResponseContent(response, headers) {
    if(typeof headers['content-type'] === 'undefined') {
        console.log(headers)
    }
    var type = (headers['content-type'] || '').split(';')[0]
    var length = 0
    try {
        length = parseInt(headers['content-length'])
    } catch (e) {}
    var result = {
        url: await response.url(),
        type: type,
        length: length,
        expires: await getExpires(headers),
    }
    // check headers and make sure we don't download too much
    if(!length || length < 100*1024*1024) {
        try {
            var buffer = await response.buffer()
            await new Promise(resolve => setTimeout(resolve, 100))
            if(result.type.includes('text/')) {
                // TODO replace with encoding from header
                result.content = buffer.toString('utf8') 
            } else {
                result.content = `data:${type};base64,${buffer.toString('base64')}`
            }
            if(result.type.includes('/html')) {
                Object.assign(result, getAllLinks(result.url, result.content))
            }
        } catch (e) {
            if(e.message.includes('Target closed')
              || e.message.includes('Response body is unavailable')
              || e.message.includes('Protocol error')) {
                console.log(`Probable tracker ${result.url}`)
            } else {
                throw e
            }
        }
    }
    return result
}

var browser
var RETRY_COUNT = 1
async function doBrowserRequest(url, readCache, storeCache, callback, retry = 0) {
    if(url === false) {
        await browser.close()
        browser = null
        return
    }
    if(!browser) browser = await puppeteer.launch()
    const page = await browser.newPage()
    await page.setRequestInterception(true)
    page.on('request', readCache)
    page.on('response', storeCache)
    var result
    try {
        result = await page.goto(url)
    } catch (e) {
        if(retry < RETRY_COUNT) {
            await doBrowserRequest(false)
            return await doBrowserRequest(url, readCache, storeCache, callback, retry+1)
        } else if(e.message.includes('ERR_NAME_NOT_RESOLVED')) {
            console.log(`Could not fetch ${url}`)
        } else {
            throw e
        }
    }
    
    if(callback) await callback(result, page)
    await page.close()
    return result
}
    
module.exports = {
    getAllLinks,
    getResponseContent,
    doBrowserRequest
}
    

#### analyze cache file?

Show some interesting information about cache files, and remove duplicates


In [ ]:
var {URL} = require('url')
var fs = require('fs')
var {findCache} = importer.import('domain crawler tools')

function analyzeCache(url) {
    var cache = findCache(url)
    if(cache.length === 0) {
        return {
            error: `No cache file found ${url}`
        }
    }
    
    var json = JSON.parse(fs.readFileSync(cache[0]))
    var domains = json.map(s => new URL(s.url).hostname)
        .filter((h, i, arr) => arr.indexOf(h) === i)
    var largeness = json.sort((a, b) => b.content.length - a.content.length)
        .slice(0, 10)
    var urls = json.map(s => s.url)
    var repeats = json.filter((s, i, arr) => i > 0 && urls.indexOf(s.url) === i)
    fs.writeFileSync(cache[0], JSON.stringify(repeats, null, 2))
    return {
        countPages: json.length,
        countCaches: cache.length,
        target: json[0].url,
        countDomains: domains.length,
        domains: domains,
        countLargest: largeness.reduce((cur, l) => cur + l.content.length, 0),
        largest10: largeness.map(l => l.url),
        repeats: json.length - repeats.length,
    }
}

module.exports = analyzeCache

#### schedule crawl domain?



In [ ]:
var google = require('googleapis');
var importer = require('../Core');
var {
    getOauthClient,
    ISODateString,
    createNewEvent,
} = importer.import([
    'convert date iso',
    'create new calendar event',
    'import google calendar api',
]);

var options = {
    calendarId: 'aws'
}

function scheduleSearch(search) {
    const parameters = {
        query: search
    }
    const newDate = new Date();
    return (typeof options.auth === 'undefined'
           ? getOauthClient(options)
           : Promise.resolve([]))
        .then(() => createNewEvent('crawl domain', JSON.stringify(parameters, null, 4), options))
}
module.exports = scheduleSearch;


### collect bookmarks pages

Convert all bookmarks to PDF and store in a JSON database for wiring up to express or my editor page.


#### introduction

One of the most irritating and terrifying things about the modern web is losing information systems depend on. 


I've experienced this, pretty much anytime some links to a Microsoft help doc without a title. They don't copy the original point of the article, or even summerize, this makes it impossible to learn. Humans are a lossy species, we lose information all the time, and perhaps it is part of evolution. I on the other hand, have gotten increasingly good at keeping information. 

What if some instruction to fix my computer suddenly disappears, should my computer become useless because of it.
What if I learn something that becomes poigently relevant sometime in the future but the link is gone.

We go one link deep, download all dependencies, and use Phantom browser to do it. It also makes a PDF so the content is preserved in different forms.

Additionally, the Bookmarks Manager in Chrome can't search page content for relevance, this is something we can fix using 


#### the code

collect all bookmarks?


In [ ]:
var path = require('path')
var importer = require('../Core')
var getBookmarksFromTakeout = importer.import('parse bookmarks file')
var ISODateString = importer.import('convert date iso')
var crawlAll = importer.import('crawl domain')
var {doBrowserRequest} = importer.import('browser crawler tools')
var {
    safeurl,
    existingCache,
    storeCache,
    readCache,
} = importer.import('domain cache tools')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/pdfs');

async function savePdf(filename, result, page) {
    const pdf = await page.pdf({ format: 'A4' })
    fs.writeFileSync(filename, pdf)
}

async function collectAllBookmarks() {
    var folders = getBookmarksFromTakeout()
    var links = folders.reduce(function flattenFolders(arr, cur) {
        if(cur.folder === 'Sad Examples') return arr
        arr.push.apply(arr, cur.links.concat(cur.children.reduce(flattenFolders, [])))
        return arr
    }, [])
    for(var i = 0; i < links.length; i++) {
        const cache = existingCache(links[i].url)
        const filename = path.join(project, safeurl(links[i].url) + '.pdf')
        
        // check if there is a recent pdf and skip
        if(fs.existsSync(filename)) continue
        await crawlAll(links[i].url, 1, cache)

        // save a pdf
        try {
            await doBrowserRequest(links[i].url, 
                                   readCache.bind(null, cache),
                                   storeCache.bind(null, cache),
                                   savePdf.bind(null, filename))
        } catch (e) {
            console.log(e)
            await doBrowserRequest(false)
        }
    }
}

module.exports = collectAllBookmarks


## html tools

TODO: move this to Languages/html.ipynb.



### find search boxes on a page



#### the code

search results as json?

In [ ]:
function searchResultsToJson(url) {
    console.log(url + ' - ' + client.requestHandler.sessionID);
    return client
        .url(url)
        .pause(2000)
        .getAllXPath({
            query: '//input[contains(@aria-label, "Search")]/@value'
            +
            '|//input[contains(@aria-label, "search")]/@value'
            +
            // yahoo
            '|//label[contains(., "Search")]/following::*//input[@type="text"]/@value' 
            +
            '|//input[contains(@class, "Search")]/@value'
            +
            // wolfram
            '|//input[contains(@name, "query")]/@value'
            +
            // duckduckgo
            '|//input[contains(@id, "search")]/@value'
            +
            // yandex
            '|//input[contains(@aria-label, "Request")]/@value',
            results: [
                '//h3|//h2|div[contains(@class, "title")]'
                +
                // ask
                '|//*[contains(@class, "item-title")]',
                {
                    name: './/text()',
                    summary: './/following-sibling::div//text()'
                }
            ]
        })
        .then(r => {
            return {
                url: url,
                query: typeof r.query === 'string'
                    ? r.query
                    : r.query[0],
                results: r.results.map(s => ({
                    name: typeof s.name === 'string'
                        ? s.name : s.name.join('\n'),
                    summary: typeof s.summary === 'string'
                        ? s.summary : s.summary.join('\n')
                }))
            };
        })
        .catch(e => {
            console.log(e)
            return {
                url: url,
                query: null,
                results: []
            }
        })
}
module.exports = searchResultsToJson;
